In [1]:
import pandas as pd
import traceback
import os,sys

sys.path.append(os.path.abspath(os.path.join('../scripts')))
from MetaCreate import MetaCreate
from AudioManipulator import AudioManipulator
from preprocess_uitls import pad_audio_files, featurize
from utils import char_map,index_map, int_sequence_to_text ,text_to_int_sequence


import librosa
import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable

    
# from keras.utils.vis_utils import plot_model


import _pickle as pickle
from numpy.lib.stride_tricks import as_strided

from Models import cnn_rnn_model,simple_rnn,model_2,bidirectional_rnn_model_gpu,bidirectional_rnn_model
# from keras.callbacks import ModelCheckpoint   

In [2]:
data=pd.read_json('../artifacts/meta.json')
data.drop(data[data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/features.pkl','rb') as tel:
    features=pickle.load(tel)
    tel.close()

In [3]:
valid_data=pd.read_json("../artifacts/valid_meta.json")
valid_data.drop(valid_data[valid_data["text"].str.contains("""[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~1234567890]""", na=False)].index,inplace=True)
with open('../artifacts/valid_features.pkl','rb') as val:
    valid_features=pickle.load(val)
    val.close()

# Augmentation


In [4]:
from preprocess_uitls import pad_audio_files,featurize,pitch_audio,shift

In [29]:
import preprocess_uitls
import importlib
importlib.reload(preprocess_uitls)

<module 'preprocess_uitls' from 'd:\\same\\Documents\\10Acadamy\\Week 4\\System\\Week-4\\scripts\\preprocess_uitls.py'>

In [5]:
data["audio"]=pad_audio_files(data["path"],98400)

In [6]:
valid_data["audio"]=pad_audio_files(valid_data["path"],98400)

In [8]:
pitched=pitch_audio(data["audio"],-2,16000)

In [9]:
pitched_valid=pitch_audio(valid_data["audio"],-1,16000)

In [10]:
pitched_features=featurize(pitched,16000)

In [11]:
with open('../artifacts/pitched_features.pickle', 'wb') as p:
        pickle.dump(pitched_features, p)

In [12]:
pitched_valid_features=featurize(pitched_valid,16000)

In [14]:
pitched=""
pitched_valid=""

In [13]:
with open('../artifacts/pitched_valid_features.pickle', 'wb') as v:
        pickle.dump(pitched_valid_features, v)

In [15]:
shifted=shift(data["audio"],"right",1,16000)

In [16]:
shifted_valid=shift(valid_data["audio"],"right",1,16000)

In [17]:
shifted_features=featurize(shifted,16000)

In [18]:
shifted_valid_features=featurize(shifted_valid,16000)

In [19]:
with open('../artifacts/shifted_features.pickle', 'wb') as v:
        pickle.dump(shifted_features, v)

In [20]:
with open('../artifacts/shifted_valid_features.pickle', 'wb') as vs:
        pickle.dump(shifted_valid_features, vs)

In [21]:
shifted=""
shifted_valid=""

In [4]:
with open('../artifacts/shifted_valid_features.pickle', 'rb') as vs:
        shifted_valid_features=pickle.load( vs)

In [5]:
with open('../artifacts/shifted_features.pickle', 'rb') as v:
        shifted_features=pickle.load( v)

In [6]:
with open('../artifacts/pitched_features.pickle', 'rb') as t:
        pitched_features=pickle.load( t)
with open('../artifacts/pitched_valid_features.pickle', 'rb') as r:
        pitched_valid_features=pickle.load( r)

In [7]:
total_text=[]
total_features=[]
total_valid_text=[]
total_valid_features=[]

In [8]:
import numpy as np

In [9]:
total_text=data["text"].tolist()+data["text"].tolist()+data["text"].tolist()
total_features=features+pitched_features+shifted_features
total_valid_text=valid_data["text"].tolist()+valid_data["text"].tolist()+valid_data["text"].tolist()
total_valid_features=valid_features+pitched_valid_features+shifted_valid_features

In [10]:
len(total_valid_features)

5970

In [25]:
data=[]
valid_data=[]

In [10]:
shifted_valid_features,shifted_features,pitched_features,pitched_valid_features=[],[],[],[]

In [4]:
import mlflow
import mlflow.tensorflow

In [5]:
from Batch import Batch

In [29]:
cnn_rnn = cnn_rnn_model(input_dim=26, # change to 13 if you would like to use MFCC features
                        filters=250,
                        kernel_size=4, 
                        conv_stride=1,
                        conv_border_mode='same',
                        units=200,output_dim=29)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 250)         26250     
_________________________________________________________________
bn_conv_1d (BatchNormalizati (None, None, 250)         1000      
_________________________________________________________________
rnn (SimpleRNN)              (None, None, 200)         90200     
_________________________________________________________________
batch_normalization (BatchNo (None, None, 200)         800       
_________________________________________________________________
time_distributed (TimeDistri (None, None, 29)          5829      
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0     

In [30]:

MODEL_NAME = "cnn_rnn_augmented"
EPOCHS=50
MINI_BATCH_SIZE = 200
FEATURES_LIST=total_features

TEXT_LIST=total_text

FEATURES_VALID_LIST=total_valid_features

TEXT_VALID_LIST=total_valid_text

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [6]:
from Train import train

In [7]:
import mlflow
import mlflow.tensorflow

In [33]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("CNN RNN Augmented Data")
train(audio_gen, input_to_softmax=cnn_rnn, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/11 03:55:25 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0c5173ae57bd4e87b1bb6c2c2348f095', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/50
140/140 [==============================] - 257s 1s/step - loss: 138.9264 - val_loss: nan
Epoch 2/50
140/140 [==============================] - 132s 945ms/step - loss: 108.4297 - val_loss: nan
Epoch 3/50
140/140 [==============================] - 134s 957ms/step - loss: 102.1505 - val_loss: nan
Epoch 4/50
140/140 [==============================] - 132s 947ms/step - loss: 98.6317 - val_loss: nan
Epoch 5/50
140/140 [==============================] - 132s 944ms/step - loss: 96.4973 - val_loss: nan
Epoch 6/50
140/140 [==============================] - 132s 941ms/step - loss: 94.6732 - val_loss: nan
Epoch 7/50
140/140 [==============================] - 133s 947ms/step - loss: 93.4806 - val_loss: nan
Epoch 8/50
140/140 [==============================] - 132s 946ms/step - loss: 92.2484 - val_loss: nan
Epoch 9/50
140/140 [==============================] - 132s 943ms/step - loss: 91.3171 - val_loss: nan
Epoch 10/50
140/140 [==============================] - 133s 947ms/step - loss: 90.

2021/08/11 05:48:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\Users\same\anaconda3\lib\site-packages\mlflow\tensorflow.py:805: UserWarning: Logging to MLflow failed: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'"


In [34]:

MODEL_NAME = "deep_augmented"
EPOCHS=20
MINI_BATCH_SIZE = 200
FEATURES_LIST=total_features

TEXT_LIST=total_text

FEATURES_VALID_LIST=total_valid_features

TEXT_VALID_LIST=total_valid_text

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [14]:
from tensorflow.keras.layers import LSTM,Bidirectional,GRU

In [50]:
deep_speech = model_2(input_dim=26,
                filters=100,
                kernel_size=4, 
                conv_stride=1,
                conv_border_mode='valid',
                units=250,
                activation='tanh',
                dropout_rate=0.6,
                cell=GRU,
                number_of_layers=4,
                output_dim=len(char_map)+1)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
layer_1_conv (Conv1D)        (None, None, 100)         10500     
_________________________________________________________________
conv_batch_norm (BatchNormal (None, None, 100)         400       
_________________________________________________________________
rnn_1 (GRU)                  (None, None, 250)         264000    
_________________________________________________________________
bt_rnn_1 (BatchNormalization (None, None, 250)         1000      
_________________________________________________________________
rnn_2 (GRU)                  (None, None, 250)         376500    
_________________________________________________________________
bt_rnn_2 (BatchNormalization (None, None, 250)         1000

In [40]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Deep Speech Augmented")
train(audio_gen, input_to_softmax=deep_speech, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/11 06:55:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'bc9d98ee181b45e4bf62e66adf4b1d4d', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/20
140/140 [==============================] - 136s 933ms/step - loss: 170.2697 - val_loss: 190.8794
Epoch 2/20
140/140 [==============================] - 129s 922ms/step - loss: 122.7602 - val_loss: 128.7204
Epoch 3/20
140/140 [==============================] - 130s 927ms/step - loss: 113.0096 - val_loss: 120.3921
Epoch 4/20
140/140 [==============================] - 129s 924ms/step - loss: 106.4057 - val_loss: 108.0783
Epoch 5/20
140/140 [==============================] - 130s 928ms/step - loss: 101.5190 - val_loss: 103.2491
Epoch 6/20
140/140 [==============================] - 130s 926ms/step - loss: 97.7856 - val_loss: 102.3560
Epoch 7/20
140/140 [==============================] - 130s 932ms/step - loss: 94.9363 - val_loss: 100.3597
Epoch 8/20
140/140 [==============================] - 132s 943ms/step - loss: 92.5748 - val_loss: 94.7952
Epoch 9/20
140/140 [==============================] - 133s 953ms/step - loss: 90.5144 - val_loss: 93.7049
Epoch 10/20
140/140 [=============

2021/08/11 07:39:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\Users\same\anaconda3\lib\site-packages\mlflow\tensorflow.py:805: UserWarning: Logging to MLflow failed: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'"


In [15]:
import Models
import importlib
importlib.reload(Models)
from Models import model_3

In [16]:

MODEL_NAME = "deep_bidirectional_augmented"
EPOCHS=20
MINI_BATCH_SIZE = 200
FEATURES_LIST=total_features

TEXT_LIST=total_text

FEATURES_VALID_LIST=total_valid_features

TEXT_VALID_LIST=total_valid_text

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [17]:
deep_speech2 = model_3(input_dim=26,
                filters=100,
                kernel_size=4, 
                conv_stride=1,
                conv_border_mode='valid',
                units=250,
                activation='tanh',
                dropout_rate=0.6,
                number_of_layers=1,
                output_dim=len(char_map)+1)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 26)]        0         
_________________________________________________________________
layer_1_conv (Conv1D)        (None, None, 100)         10500     
_________________________________________________________________
conv_batch_norm (BatchNormal (None, None, 100)         400       
_________________________________________________________________
bidirectional (Bidirectional (None, None, 500)         528000    
_________________________________________________________________
bt_rnn_1 (BatchNormalization (None, None, 500)         2000      
_________________________________________________________________
time_distributed (TimeDistri (None, None, 29)          14529     
_________________________________________________________________
softmax (Activation)         (None, None, 29)          0     

In [18]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Deep Speech Augmented")
train(audio_gen, input_to_softmax=deep_speech2, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/11 08:15:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e647f96f593a42afb18c522e5d1d72a3', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/20
140/140 [==============================] - 117s 756ms/step - loss: 168.8782 - val_loss: 158.7168
Epoch 2/20
140/140 [==============================] - 104s 743ms/step - loss: 130.6323 - val_loss: 125.5214
Epoch 3/20
140/140 [==============================] - 106s 761ms/step - loss: 120.8133 - val_loss: 116.1417
Epoch 4/20
140/140 [==============================] - 103s 734ms/step - loss: 108.7661 - val_loss: 113.8748
Epoch 5/20
140/140 [==============================] - 104s 741ms/step - loss: 103.9581 - val_loss: 108.4986
Epoch 6/20
140/140 [==============================] - 104s 745ms/step - loss: 100.4474 - val_loss: 101.1788
Epoch 7/20
140/140 [==============================] - 105s 748ms/step - loss: 97.9822 - val_loss: 101.6945
Epoch 8/20
140/140 [==============================] - 102s 730ms/step - loss: 95.8176 - val_loss: 103.2539
Epoch 9/20
140/140 [==============================] - 108s 774ms/step - loss: 94.2637 - val_loss: 94.2850
Epoch 10/20
140/140 [===========

2021/08/11 08:50:36 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "D:\Users\same\anaconda3\lib\site-packages\mlflow\tensorflow.py:805: UserWarning: Logging to MLflow failed: module 'keras.utils.generic_utils' has no attribute 'populate_dict_with_module_objects'"


In [19]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Deep Speech Augmented")
train(audio_gen, input_to_softmax=deep_speech2, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

2021/08/11 08:50:41 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '98ddf6d3e0464ac2a311f868a9746aec', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/20
140/140 [==============================] - 112s 776ms/step - loss: 83.9986 - val_loss: 89.0362
Epoch 2/20
140/140 [==============================] - 102s 729ms/step - loss: 83.4881 - val_loss: 91.5093
Epoch 3/20
140/140 [==============================] - 100s 713ms/step - loss: 83.1177 - val_loss: 93.2569
Epoch 4/20
140/140 [==============================] - 131s 941ms/step - loss: 82.6347 - val_loss: 92.3615
Epoch 5/20
140/140 [==============================] - 176s 1s/step - loss: 82.0475 - val_loss: 90.0516
Epoch 6/20
140/140 [==============================] - 107s 765ms/step - loss: 81.7676 - val_loss: 89.1217
Epoch 7/20
140/140 [==============================] - 98s 704ms/step - loss: 81.3305 - val_loss: 90.2516
Epoch 8/20
140/140 [==============================] - 97s 691ms/step - loss: 81.1707 - val_loss: 87.3265
Epoch 9/20
140/140 [==============================] - 97s 693ms/step - loss: 80.8007 - val_loss: 90.2307
Epoch 10/20
140/140 [==============================]

In [12]:
deep_speech_4 = model_2(input_dim=26,
                filters=100,
                kernel_size=4, 
                conv_stride=1,
                conv_border_mode='valid',
                units=250,
                activation='sigmoid',
                dropout_rate=0.7,
                number_of_layers=4,
                output_dim=len(char_map)+1)

In [10]:

MODEL_NAME = "deep_speech"
EPOCHS=40
MINI_BATCH_SIZE = 250
FEATURES_LIST=features

TEXT_LIST=data["text"].tolist()

FEATURES_VALID_LIST=valid_features

TEXT_VALID_LIST=valid_data["text"].to_numpy()

audio_gen = Batch(FEATURES_LIST,FEATURES_VALID_LIST,TEXT_LIST,TEXT_VALID_LIST,MINI_BATCH_SIZE)

In [11]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("Deep Speech")
train(audio_gen, input_to_softmax=deep_speech_4, model_name=MODEL_NAME, epochs=EPOCHS, minibatch_size=MINI_BATCH_SIZE)

mlflow.end_run()

D:\Users\same\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021/08/11 09:33:59 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'c2f046e1474e42f393191a26d3bc72b4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/40


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[250,250] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model_1/rnn_3/while/body/_452/model_1/rnn_3/while/gru_cell_2/ones_like}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[model_1/ctc/Log/_380]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[250,250] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node model_1/rnn_3/while/body/_452/model_1/rnn_3/while/gru_cell_2/ones_like}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions fo